In [ ]:
import geemap
import ee
import math

# 设置代理（根据你的网络环境）
geemap.set_proxy(port=10809)

# 认证和初始化
ee.Authenticate()
ee.Initialize(project='ee-mrwurenzhe')

# 定义时空范围
start_date = '2024-06-01'
end_date = '2024-09-30'
region = ee.Geometry.Rectangle([70, 10, 140, 60])  # 东亚范围（min_lon, min_lat, max_lon, max_lat）

# 获取 ERA5 月平均数据
era5_monthly = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
    .filterDate(start_date, end_date) \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])

# 计算时间平均（所有月份的平均）
u_mean = era5_monthly.select('u_component_of_wind_10m').mean()
v_mean = era5_monthly.select('v_component_of_wind_10m').mean()

# 计算风速和风向
def calculate_wind_properties(image_u, image_v):
    # 风速 = sqrt(u^2 + v^2)
    wind_speed = image_u.pow(2).add(image_v.pow(2)).sqrt().rename('wind_speed')
    
    # 风向（弧度）= atan2(v, u)，转换为气象风向（单位：度）
    wind_direction = image_v.atan2(image_u) \
        .multiply(180 / math.pi) \
        .add(180) \
        .rename('wind_direction')  # 转换为 0-360 度，风的来源方向
    
    return wind_speed, wind_direction

# 计算风速和风向
wind_speed, wind_direction = calculate_wind_properties(u_mean, v_mean)

In [10]:
# 可视化参数
wind_speed_vis_params = {
    'min': 0,
    'max': 15,
    'palette': ['blue', 'green', 'yellow', 'red']
}

wind_direction_vis_params = {
    'min': 0,
    'max': 360,
    'palette': ['purple', 'blue', 'green', 'yellow', 'red']
}

u_vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}

v_vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}


# 创建地图
Map = geemap.Map()
Map.centerObject(region, 4)

# 添加风速和风向图层
Map.addLayer(wind_speed, wind_speed_vis_params, 'ERA5 Mean Wind Speed (2015-2024)')
Map.addLayer(wind_direction, wind_direction_vis_params, 'ERA5 Mean Wind Direction (2015-2024)')
Map.addLayer(region, {'color': 'yellow'}, 'ROI')

# 显示地图
Map

Map(center=[35.972644113075546, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [11]:
import geemap

# Define output path
output_file = "ERA5_Mean_Wind_Direction_Speed_q3.tif"
geemap.download_ee_image(
    image=wind_direction.addBands(wind_speed),
    filename=output_file,
    region=region,
    scale=11132,  # ERA5-Land resolution
    crs='EPSG:4326'
)


output_file = "ERA5_Mean_Wind_uv_mean_q3.tif"
geemap.download_ee_image(
    image=u_mean.addBands(v_mean),  
    filename=output_file,
    region=region,
    scale=11132,  # ERA5-Land resolution
    crs='EPSG:4326'
)

ERA5_Mean_Wind_Direction_Speed_2015-2024.tif: |          | 0.00/6.15M (raw) [  0.0%] in 00:00 (eta:     ?)

ERA5_Mean_Wind_uv_mean_2015-2024.tif: |          | 0.00/6.15M (raw) [  0.0%] in 00:00 (eta:     ?)